<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv1D,Conv2D, BatchNormalization, Dropout, Activation, Softmax, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam,RMSprop
import numpy as np
import pandas as pd
import os
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import random
import pprint
%matplotlib inline
# from google.colab import drive
# drive.mount('/content/drive')
from sklearn.model_selection import train_test_split

classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [ ]:
tf.__version__

'2.4.1'

In [2]:
!pip install wandb
!wandb login

wandb: Currently logged in as: narendv (use `wandb login --relogin` to force relogin)


In [4]:
import wandb
wandb.init(project="CS6910-assg2", entity="narendv")

In [18]:
def load_train_data(dir, augment= True,batch=128):
  seed=42
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    samplewise_center = 0,
    horizontal_flip = True,
    rotation_range = 30,
    validation_split = 0.1)
  val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    samplewise_center = 0,
    validation_split = 0.1)
  train_aug_dataset = train_datagen.flow_from_directory(
    dir,
    target_size=(150,150),
    batch_size=batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = 'training',
    seed = seed)

  train_dataset = val_datagen.flow_from_directory(
    dir,
    target_size=(150,150),
    batch_size=batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = 'training',
    seed = seed)
  
  val_dataset = val_datagen.flow_from_directory(
    dir,
    target_size=(150,150),
    batch_size=batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = 'validation',
    seed = seed)
  
  return train_dataset, val_dataset

In [22]:
learn_rate = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=1000,
    decay_rate=0.9)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)


In [15]:
def CNN_wo_BN(filter_matrix, kernel_matrix, activation_matrix, fdropout=0, dense_no=256, learning_rate=0.001):

  # Input layer
  X_input = keras.Input(shape=(150,150,3))

  # Layer 1
  X = Conv2D(filter_matrix[0], kernel_matrix[0])(X_input)
  X = Activation(activation_matrix[0])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 2
  X = Conv2D(filter_matrix[1], kernel_matrix[1])(X)
  X = Activation(activation_matrix[1])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 3
  X = Conv2D(filter_matrix[2], kernel_matrix[2])(X)
  X = Activation(activation_matrix[2])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 4
  X = Conv2D(filter_matrix[3], kernel_matrix[3])(X)
  X = Activation(activation_matrix[3])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 5
  X = Conv2D(filter_matrix[4], kernel_matrix[4])(X)
  X = Activation(activation_matrix[4])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  X = Flatten()(X)
  X = Dense(dense_no,activation=activation_matrix[5])(X)
  X = Dropout(fdropout)(X)
  X = Dense(10,activation='softmax')(X)
  
  model=Model(inputs=X_input,outputs=X)
  model.compile(optimizer=Adam(learn_rate), loss='categorical_crossentropy',  metrics=['accuracy'])

  return model

In [16]:
def CNN_w_BN(filter_matrix, kernel_matrix, activation_matrix, fdropout=0, dense_no=256, learning_rate=0.001):

  # Input layer
  X_input = keras.Input(shape=(150,150,3))

  # Layer 1
  X = Conv2D(filter_matrix[0], kernel_matrix[0])(X_input)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[0])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 2
  X = Conv2D(filter_matrix[1], kernel_matrix[1])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[1])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 3
  X = Conv2D(filter_matrix[2], kernel_matrix[2])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[2])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 4
  X = Conv2D(filter_matrix[3], kernel_matrix[3])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[3])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  # Layer 5
  X = Conv2D(filter_matrix[4], kernel_matrix[4])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[4])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)

  X = Flatten()(X)
  X = Dense(dense_no,activation=activation_matrix[5])(X)
  X = Dropout(fdropout)(X)
  X = Dense(10,activation='softmax')(X)
  
  model=Model(inputs=X_input,outputs=X)
  model.compile(optimizer=Adam(learn_rate), loss='categorical_crossentropy',  metrics=['accuracy'])

  return model

In [10]:
def model_choose(highest_filter, filter_type, kernel_matrix, activation_matrix, dropout=0, dense_no=256, batch_norm=True, learning_rate=0.001):
  if filter_type == 'constant':
    filter_matrix = [filter_start]*4
  elif filter_type == 'doubling':
    filter_matrix = [highest_filter/16, highest_filter/8, highest_filter/4, highest_filter/2, highest_filter]
  elif filter_type == 'halving':
    filter_matrix = [highest_filter, highest_filter/2, highest_filter/4, highest_filter/8, highest_filter/16]
  # wandb.log({"Filters": filter_matrix})

  if batch_norm == True:
    model = CNN_w_BN(filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, learning_rate)
  else:
    model = CNN_wo_BN(filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, learning_rate)
  
  return model


In [23]:
# Checking all above functions
X=tf.random.normal((227,227,3))
# filter_matrix = [32, 64, 256, 512, 1024]
kernel_matrix = [(3,3)]*5
activation_matrix = ['relu']*6;
model = model_choose(1024, 'doubling', kernel_matrix, activation_matrix, batch_norm=False)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 254, 254, 64)      1792      
_________________________________________________________________
activation_10 (Activation)   (None, 254, 254, 64)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 125, 125, 128)     73856     
_________________________________________________________________
activation_11 (Activation)   (None, 125, 125, 128)     0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 62, 62, 128)       0   

In [ ]:
@tf.custom_gradient
def GBRelu(x):
  def gradient(dy):
    grad = tf.cast(dy>0,"float32")*tf.cast(x>0, "float32")*dy
    return  grad
  return tf.nn.relu(x), gradient
# Here model is the best model
GBModel = Model(inputs = [model.inputs],outputs = [model.get_layer(index=8).output]) # Here the index is to be checked
layer_dict = [layer for layer in gb_model.layers[1:] if hasattr(layer,'activation')]
for layer in layer_dict:
  if layer.activation == tf.keras.activations.relu:
    layer.activation = GBRelu

with tf.GradientTape() as tape:
  inputs = tf.cast(X_input, tf.float32)
  tape.watch(inputs)
  outputs = GBModel(inputs)

grads = tape.gradient(outputs,inputs)[0]

In [9]:
sweep_config = {
    'method': 'grid'
    }
parameters_dict = {
    'highest_filter':{
        'values': [256]
      },
    'kernel_matrix': {
        'values': [0]
      },
    'learn_rate': {
        'values': [1e-3]
      },
    'epochs': {
        'values': [5,10]
      },
    'batch_size': {
        'values': [256]
      },
    'dropout': {
        'values': [0.2]
      },
    'batch_normalization': {
        'values': [True]
      },
    'dense_number': {
          'values': [512]
      },
    'augmentation':{
        'values' : [True]
    }
}
sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)

def training_sweep(config=None):
    with wandb.init(config=config):
        config = wandb.config
        kernels = [[(3,3)]*5]
        global K
        K = kernels[config.kernel_matrix]
        activation_matrix = ['relu']*6
        model = model_choose(config.highest_filter, 'doubling', K, activation_matrix, config.dropout, config.dense_number, config.batch_normalization, config.learn_rate)
        train, val = load_train_data('/content/drive/MyDrive/inaturalist_12K/train', config.augmentation,config.batch_size)
        history = model.fit(train, 
                            epochs=config.epochs,
                            validation_data = val,callbacks=callback
                            )
        wandb.log(history.history)

{'method': 'grid',
 'parameters': {'augmentation': {'values': [True]},
                'batch_normalization': {'values': [True, False]},
                'batch_size': {'values': [256]},
                'dense_number': {'values': [512]},
                'dropout': {'values': [0.2]},
                'epochs': {'values': [5, 10]},
                'highest_filter': {'values': [1024]},
                'kernel_matrix': {'values': [0]},
                'learn_rate': {'values': [0.001]}}}


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910-assg2")
wandb.agent(sweep_id, training_sweep)

In [20]:
k = [(3,3)]*5
activation_matrix = ['relu']*6

train, val = load_train_data('/content/drive/MyDrive/inaturalist_12K/train',True, 128)
model = model_choose(256, 'doubling', k, activation_matrix, 0, 512, True, 0.001)

model.fit(train, 
          epochs=5,
          validation_data = val,
          )

Found 9018 images belonging to 10 classes.
Found 1001 images belonging to 10 classes.
Epoch 1/5
71/71 [==============================] - 1391s 20s/step - loss: 2.4553 - accuracy: 0.2098 - val_loss: 2.4281 - val_accuracy: 0.1029
Epoch 2/5
71/71 [==============================] - 176s 2s/step - loss: 1.9294 - accuracy: 0.3145 - val_loss: 2.4226 - val_accuracy: 0.1518
Epoch 3/5
71/71 [==============================] - 176s 2s/step - loss: 1.8629 - accuracy: 0.3450 - val_loss: 2.3838 - val_accuracy: 0.1768
Epoch 4/5
71/71 [==============================] - 172s 2s/step - loss: 1.7781 - accuracy: 0.3818 - val_loss: 2.6401 - val_accuracy: 0.1329
Epoch 5/5
71/71 [==============================] - 171s 2s/step - loss: 1.7321 - accuracy: 0.3896 - val_loss: 2.1608 - val_accuracy: 0.2438


In [24]:

model.fit(train, 
          epochs=10,
          validation_data = val, callbacks=[callback]
          )

Epoch 1/10
71/71 [==============================] - 174s 2s/step - loss: 2.0388 - accuracy: 0.2627 - val_loss: 2.2891 - val_accuracy: 0.1718
Epoch 2/10
71/71 [==============================] - 169s 2s/step - loss: 2.0038 - accuracy: 0.2782 - val_loss: 2.2991 - val_accuracy: 0.2078
Epoch 3/10
71/71 [==============================] - 176s 2s/step - loss: 1.9761 - accuracy: 0.2911 - val_loss: 2.3729 - val_accuracy: 0.2068
Epoch 4/10
71/71 [==============================] - 175s 2s/step - loss: 1.9480 - accuracy: 0.3000 - val_loss: 2.0793 - val_accuracy: 0.2597
Epoch 5/10
71/71 [==============================] - 172s 2s/step - loss: 1.9367 - accuracy: 0.3069 - val_loss: 2.5163 - val_accuracy: 0.2228
Epoch 6/10
71/71 [==============================] - 170s 2s/step - loss: 1.9140 - accuracy: 0.3158 - val_loss: 2.2765 - val_accuracy: 0.2517
Epoch 7/10
71/71 [==============================] - 170s 2s/step - loss: 1.8869 - accuracy: 0.3247 - val_loss: 2.4481 - val_accuracy: 0.1978
